In [1]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [2]:
# %cd /content/drive/MyDrive/AI-bot

In [24]:
import numpy as np
import pandas as pd
from hazm import *
from collections import Counter

In [12]:
df = pd.read_csv("base_responses.csv", encoding="utf-8").dropna()
df.tail()

,Class1-Q,Class1-W,Class2-Q,Class2-W,Class3-Q,Class3-W,Class4-Q,Class4-W
80,کی دوباره بارون می آید؟ // چرا خونه انقدر سرده...,بارون - تهران - برف - مه,ماه رمضون تهران زودتر اذان میشه یا اصفهان ؟\nز...,اذان - ماه رمضون - اصفهان - تهران- زمستون,الان سئول ساعت چنده؟\nالان شیلی ساعت چنده؟\nاگ...,سئول / الان /شیلی / یه روز /آفریقا/ ایران,چند روز تا کریسمس مونده؟\nروز جهانی خبرنگار چه...,کریسمس/ خبرنگار/شهریور/مهر/سپتامبر
81,آخرین باری که بارون اومد کی بود؟ // هوا قراره ...,آخرین بار - بارون // هوا - سرد // فقط - ابری -...,اذان صبح چه ساعتیه؟ // تا غروب چقدر مونده؟ // ...,اذان - صبح - ساعت // غروب - چقدر - مونده // اذ...,چقدر به ساعت دو مونده؟ // نه ساعت دیگه ساعت چن...,چقدر - ساعت - دو // نه - ساعت - دیگه // وقت - ...,روز دانشجو چه روزیه؟ // امسال محرم کی شروع میش...,روز - دانشجو // امسال - محرم - کی // چند - روز...
82,امروز هوا چطوره؟\nهوا چند درجست؟\nبنظرت از سال...,هوا درجه هفته سال پیش,ساعت چند اذان صبح میگن؟\nفاصله اذان تهران با ا...,اذان، صبح، نیمه شب شرعی، ساعت,الان ساعت چنده؟\nساعت تو واشینگتن چنده؟\nالان ...,ساعت، روز، شب، اختلاف، جلوتر، عقب‌تر,امروز چندمه؟\nهفته بعد به چه مناسبتی تعطیله؟\n...,امروز، روز، مناسبت
83,آب و هوای هفته آینده رفسنجان چطور است؟\nمیزان ...,آب و هوا-رفسنجان-هفته آینده//بارش باران-میلیمت...,اذان صبح تهران امروز چه زمانی است؟\nنیمه شب شر...,اذان صبح-تهران-امروز//نیمه شب شرعی-کرمان,اختلاف ساعتی تهران و کالیفرنیا چقدر است؟\nالان...,اختلاف ساعت-تهران-کالیفرنیا//الان-مسکو-شب یا ر...,چند روز تا شب یلدا باقی مانده؟\nروز معلم چه تا...,شب یلدا//روز معلم-چه تاریخی//تاریخ تولد-شما-چه...
85,هوا چطوره؟,فردا-برف-بارون-نم-سیگار-قهوه,کی اذان میگه؟,اینجا-کی-غروب,کی بیام؟,میای-کی-ساعت-دیر,امروز چه روزیه؟,شب چله - روز پرستار


In [41]:
unused_chars = ['-', '،', '_', '\n', '//', '/', '\u200c', 'ـ']
unused_words = ['و', 'ف', 'آل', 'قهوه', 'فقط', 'بار', 'بریم',
                'فقط', 'غیرقابل', 'نظر', 'یک', 'آمریکا', 'رفسنجان',
                'بهترین', 'آخرین', 'کجا', 'همدان', 'تورنتو',
                'هند', 'متنوع', 'کشور', 'خیلی', 'اینجا',
                'چطور', 'کی', 'قناری', 'چنوبی', 'نیم', 'بعد', 
                'عجیبی', 'طرز', 'بندر', 'پاییزی', 'زندگی', 'چیه', 
                'چطوره', 'محل', 'رانندگی', 'احتمال', 'اولین', 'کیری', 
                'کالیفرنیا', 'آخر', 'ادامه', 'کلانشهر', 'سفر', 'دم', 'مالدیو', 
                'تنفر', 'جمعه', 'تبریز', 'آذرماه', 'اکنون', 'گیلان', 'البرز', 'ساری', 
                'دو', 'غروب', 'ظهر', 'بهتر', 'بینی', 'ایده', 'آزاد', 'دانشگاه', 
                'ترافیک', 'کمتر', 'قشنگ', 'قزوین', 'شوشتر', 'کم', 'قم', 
                'پاکیزه', 'بر', 'کرمان', 'کلی', 'روشن', 'داشتن', 'دوست', 'عالی', 
                'شنبه', 'سه', 'عشق', 'آتی', 'بیشترین', 'حالا', 'طولانیترین', 'سنگینتر', 
                'روی', 'پیاده', 'پودر', 'دربندسر', 'شمشک', 'دیزین', 
                'آیا', 'ترین', 'زنجان', 'دونفره', '۱۰', 'بیرون', 'جزایر', 'گذشته', 
                'یا', 'اهواز', 'نوشهر', 'رشت', 'تحمل', 'رو', 'لباس', 
                'سیگار', 'خوب', 'تا', 'برلین', 'نظرت', 'مشهد', 'الان', 'آینده', 
                'پیش', 'دیروز', 'امسال', 'ایران', 'پارسال', 'هفته', 'امروز', 'فردا', 'تهران']

In [ ]:
norm = Normalizer()

In [ ]:
words = df["Class1-W"].tolist()
words1 = []
for s in words:
    t = norm.normalize(s)
    for c in unused_chars:
        t = t.replace(c, ' ')
    t = ' '.join(t.split())
    words1.append(t)
# print(words1)

all_words = [w for w in ' '.join(words1).split() if w not in unused_words]
ranking = Counter(all_words).most_common()
print(ranking), len(ranking)

In [45]:
words = df["Class1-W"].tolist()
words1 = []
for s in words:
    t = norm.normalize(s)
    for c in unused_chars:
        t = t.replace(c, ' ')
    t = ' '.join(t.split())
    words1.append(t)
# print(words1)

all_words = [w for w in ' '.join(words1).split() if w not in unused_words]
ranking = Counter(all_words).most_common()
print(ranking), len(ranking)

[('هوا', 81), ('برف', 19), ('سرد', 17), ('باران', 16), ('بارون', 14), ('آب', 12), ('درجه', 11), ('گرم', 9), ('ابری', 8), ('آفتابی', 7), ('بارانی', 7), ('دما', 6), ('سال', 6), ('هواشناسی', 5), ('روز', 5), ('آلودگی', 4), ('بارش', 3), ('ابر', 3), ('وضعیت', 3), ('کره', 3), ('بارونی', 3), ('زمین', 2), ('آفتاب', 2), ('سردتر', 2), ('آلوده', 2), ('سرعت', 2), ('باد', 2), ('اب', 2), ('خشک', 2), ('طوفانی', 2), ('پاییز', 2), ('تابستان', 2), ('مه', 2), ('میانگین', 2), ('طوفان', 2), ('شرجی', 2), ('سردترین', 2), ('شب', 1), ('زمستان', 1), ('ارتفاع', 1), ('رگبار', 1), ('مرطوب', 1), ('آسمان', 1), ('معتدل', 1), ('برفی', 1), ('شاخص', 1), ('رطوبت', 1), ('دمای', 1), ('فصل', 1), ('الوده', 1), ('بارونیه؟', 1), ('شهر', 1), ('سیل', 1), ('سردی', 1), ('جنوبی', 1), ('ببارد', 1), ('اخیر', 1), ('افزایش', 1), ('وهوا', 1), ('استوایی', 1), ('میلیمتر', 1), ('گزارش', 1), ('سامانه', 1), ('بارشی', 1), ('پایدار', 1), ('امریکا', 1), ('خسارت', 1), ('نم', 1)]


(None, 68)